In [1]:
from matmul import *

ELAPSED TIME TORCH 0.024219274520874023
ELAPSED TIME BASIC MATMUL 26.95095658302307
ELAPSED TIME ELEMENTWISE MATMUL 0.4111135005950928
ELAPSED TIME BROADCAST MATMUL 0.005246877670288086
ELAPSED TIME EINSUM MATMUL 0.0007429122924804688
ELAPSED TIME @ MATMUL 0.00012087821960449219


In [2]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
EPSILON = 1e-3

In [3]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path) as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

x_train, y_train, x_valid, y_valid = get_data()

In [4]:
def normalize(data, mean, std):
    return (data - mean) / std

In [5]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [6]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [7]:
train_mean, train_std = x_train.mean(), x_train.std()
assert train_mean.abs() < EPSILON
assert (train_std - 1).abs() < EPSILON

# Basic 1 hidden layer architecture

In [8]:
num_hidden = 50
num_inputs = 784
w1 = torch.randn(num_inputs, num_hidden) / math.sqrt(num_inputs)
b1 = torch.zeros(num_hidden)
w2 = torch.randn(num_hidden, 1) / math.sqrt(num_hidden)
b2 = torch.zeros(1)

In [9]:
def linear_layer(X, w, b):
    return X@w + b

In [10]:
def relu(x):
    return torch.clamp_min(x, 0.)

In [11]:
first_lay = relu(linear_layer(x_valid, w1, b1))

In [12]:
first_lay.mean(), first_lay.std()

(tensor(0.4472), tensor(0.6187))

It's bad to have standard deviation halve each layer. That would mean that at the last layer the std is very small. We'll solve it by changing the initialization of the weights

# Kaiming initialization:

In [13]:
num_hidden = 50
num_inputs = 784
w1 = torch.randn(num_inputs, num_hidden) * math.sqrt(2/num_inputs)
b1 = torch.zeros(num_hidden)
w2 = torch.randn(num_hidden, 1) * math.sqrt(2/num_hidden)
b2 = torch.zeros(1)

In [14]:
first_lay = relu(linear_layer(x_valid, w1, b1))

In [15]:
first_lay.mean(), first_lay.std()

(tensor(0.5595), tensor(0.8280))

Comparing to actual torch's implementation

In [16]:
from torch.nn import init
w1 = torch.zeros(num_inputs, num_hidden)
init.kaiming_normal_(w1, mode='fan_out') # Preserve magnitude in backwards pass
# That happens because pytorch does a transpose matrix multiply in standard linear layer.

first_lay_kaim = relu(linear_layer(x_valid, w1, b1))
first_lay_kaim.mean(), first_lay_kaim.std()

(tensor(0.5081), tensor(0.7832))

Seems right.

The mean is still 0.5, which makes sense. However, what if we made a new relu that subtracts 0.5 from the mean?

In [17]:
def relu(x):
    return torch.clamp_min(x, 0.) - 0.5

In [18]:
first_lay_kaim = relu(linear_layer(x_valid, w1, b1))
first_lay_kaim.mean(), first_lay_kaim.std()

(tensor(0.0081), tensor(0.7832))

In [19]:
def model(x):
    X = linear_layer(x, w1, b1)
    X = relu(X)
    X = linear_layer(X, w2, b2)
    return X

In [20]:
assert model(x_valid).shape == (x_valid.shape[0],1)

# MSE
Even though it is not suitable for multi-class classification it is an easy function to code so I'll use it just for testing

In [21]:
def mse(y_pred, y_true):
    return (y_pred.squeeze(1) - y_true).pow(2).mean()

In [22]:
y_train, y_valid = y_train.float(), y_valid.float()
preds = model(x_train)
preds.shape, y_train.shape

(torch.Size([50000, 1]), torch.Size([50000]))

In [23]:
mse(preds, y_train)

tensor(33.0932)

# Backwards pass

In [24]:
def mse_backwards(inp, out):
    inp.g = 2. * (inp.squeeze() - out).unsqueeze(-1) / inp.shape[0]

In [25]:
mse_backwards(preds, y_train)
print(preds.g.shape)

torch.Size([50000, 1])


In [26]:
def linear_layer_backwards(inp, out, w, b):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(dim=0)
    b.g = out.g.sum(dim=0)

In [27]:
def relu_backwards(inp, out):
    inp.g = (inp > 0).float() * out.g

In [28]:
def forward_and_backward(inp, targ):
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    loss = mse(out, targ)
    
    mse_backwards(out, targ)
    linear_layer_backwards(l2, out, w2, b2)
    relu_backwards(l1, l2)
    linear_layer_backwards(inp, l1, w1, b1)

In [29]:
forward_and_backward(x_train, y_train)

Saving results to compare to PyTorch's backpropagation

In [30]:
w1g = w1.g.clone()
b1g = b1.g.clone()
w2g = w2.g.clone()
b2g = b2.g.clone()
inpg = x_train.g.clone()

In [31]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [32]:
def forward(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    l3 = l2 @ w22 + b22
    return mse(l3, targ)

In [33]:
loss = forward(xt2, y_train)

In [34]:
loss.backward()

In [35]:
def near_enough(a, b):
    assert torch.allclose(a, b, rtol=1e-3, atol=1e-5)

In [36]:
near_enough(xt2.grad, inpg)
near_enough(w12.grad, w1g)
near_enough(b12.grad, b1g)
near_enough(w22.grad, w2g)
near_enough(b22.grad, b2g)

All gradients seem to be correct.

# Refactoring: make layers become classes

In [37]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g

In [38]:
class Linear_layer():
    def __init__(self, w, b):
        self.w = w
        self.b = b
    
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(dim=0)
        self.b.g = self.out.g.sum(dim=0)

In [39]:
class MSE():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze(-1) - self.targ).pow(2).mean()
        return self.out

    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.inp.shape[0]

In [40]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Linear_layer(w1, b1), Relu(), Linear_layer(w2, b2)]
        self.loss = MSE()
        
    def __call__(self, x, y):
        for lay in self.layers:
            x = lay(x)
        return self.loss(x, y)
    
    def backward(self):
        self.loss.backward()
        for lay in reversed(self.layers):
            lay.backward()
            

In [41]:
# Reset gradients
w1.g, b1.g, w2.g, b2.g = [None]*4

model = Model(w1, b1, w2, b2)

In [42]:
%time loss = model(x_train, y_train)

CPU times: user 204 ms, sys: 92 ms, total: 296 ms
Wall time: 36.2 ms


In [43]:
%time model.backward()

CPU times: user 6.47 s, sys: 5.52 s, total: 12 s
Wall time: 1.57 s


In [44]:
near_enough(x_train.g, inpg)
near_enough(w1.g, w1g)
near_enough(b1.g, b1g)
near_enough(w2.g, w2g)
near_enough(b2.g, b2g)

Seems good, but we're hurting the DRY (Don't Repeat Yourself) principle too much. Let's refactor even more

In [54]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self):
        raise Exception("Not yet implemented")
    
    def backward(self):
        self.bwd(self.out, *self.args)

In [55]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.) - 0.5
    
    def bwd(self, out, inp):
        inp.g = (inp > 0).float() * out.g

In [56]:
class Linear_layer(Module):
    def __init__(self, w, b):
        self.w = w
        self.b = b
        
    def forward(self, inp):
        return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(dim=0)

In [57]:
class MSE(Module):
    def forward(self, inp, targ):
        return (inp.squeeze() - targ).pow(2).mean()
    
    def bwd(self, out, inp, targ): 
        inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [58]:
class Model():
    def __init__(self):
        self.layers = [Linear_layer(w1, b1), Relu(), Linear_layer(w2, b2)]
        self.loss = MSE()
    
    def __call__(self, x, targ):
        for layer in self.layers:
            x = layer(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for layer in reversed(self.layers):
            layer.backward()

In [62]:
# Reset gradients
w1.g, b1.g, w2.g, b2.g = [None]*4

model = Model()

In [63]:
%time loss = model(x_train, y_train)

CPU times: user 220 ms, sys: 120 ms, total: 340 ms
Wall time: 59.4 ms


In [64]:
%time model.backward()

CPU times: user 368 ms, sys: 392 ms, total: 760 ms
Wall time: 121 ms


In [65]:
near_enough(x_train.g, inpg)
near_enough(w1.g, w1g)
near_enough(b1.g, b1g)
near_enough(w2.g, w2g)
near_enough(b2.g, b2g)

In [66]:
from torch import nn

In [76]:
def mse(y_pred, y_true):
    return (y_pred.squeeze() - y_true).pow(2).mean()

In [81]:
class Model(nn.Module):
    def __init__(self, n_in, n_hid, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, n_hid), nn.ReLU(), nn.Linear(n_hid, n_out)]
        self.loss = mse
    
    def __call__(self, x, y):
        for layer in self.layers:
            x = layer(x)
        return self.loss(x.squeeze(), y)

In [82]:
n,m = x_train.shape
nh = 50

In [83]:
model = Model(m, nh, 1)

In [84]:
%time loss = model(x_train, y_train)

CPU times: user 172 ms, sys: 140 ms, total: 312 ms
Wall time: 48.1 ms


In [86]:
%time loss.backward()

CPU times: user 332 ms, sys: 184 ms, total: 516 ms
Wall time: 75.5 ms
